GPU Memory costs analysis for _GraphRNN_:
1. The default setting to train the model contains 100 grids with size $m * n$, where $m, n \in [10, 19]$.

2. For each graph, it randomly chooses a start point and do _Breadth-First-Search(BFS)_, then it uses the BFS order as the order of nodes of graphs. Based on that order, it builds the adjacent matrix for each graph.

3. According to the properties of BFS, each node can only reach node in the next layer in the BFS, as shown in the following figure, nodes from the same layer are in the same color.

![p1.png](p1.png)

Therefore, instead of using the original adjacent matrix, each node only considers the connection to 40 (default setting) nodes before it. Thus, the adjacent matrix used to represent a graph becomes:
![image.png](p2.jpg)

GraphRNN trains the model with 3000 epochs, each epoch contains 32 batches and each batch has 32 graphs.

Some main steps inducing memory costs:

In pytorch, GPU memory costs are divided into two parts: 1._allocated memory_; 2. _cached memory_

1.Allocated memory is the GPU memory currently occupied by tensors; 2.Cached memory is the GPU memory occupied by pytorch, it includes allocated memory and memory currently kept by pytorch for fast allocating to the tensors in the future. So in the remaining part of this report, I will show both allocated memory and cached memory.

- Decalim two RNN used in this work whose name is ___rnn___ and ___output___,___rnn___ is for graph level and ___output___ is for edge level: allc:1.57MB, cach: 3.5MB
- Initialize the hidden state:  allc:1.63MB, cach: 2.5MB
- prepare the initial input for ___rnn___ and ___output___: allc: 6.88MB cach: 7.75MB
- Graph level RNN ___rnn___:
![image.png](p3.jpg)
    - One fully connected layer and ReLU: allc: 11.64MB, cach: 12.5MB
    - Remove padded rows and combine all graphs into one 2D tensor: 11.01MB, cach: 15.25MB
    - 4 layer _Gated Recurrent Unit_ (GRU): allc:106.95MB, cach: 114.5MB
    - Restore the 2D tensor to different graphs: allc: 111.7MB, cach: 119.25MB
    - two fully connected layer and ReLU: allc: 127.41MB, cach: 134.75MB